# Notebook for developing and testin the Itinerary Code Pilot

In [ ]:
import requests
import json
import os
import openai
import ls_nb_utils
import pandas as pd

In [ ]:
openai.api_key  = os.environ['OPENAI_API_KEY']
client = openai.OpenAI()


In [ ]:
ls_app_url =  'https://pocketverse.herokuapp.com/LS_API'
ls_app_username = os.environ['LS_USERNAME']
ls_app_password = os.environ['LS_PASSWORD']
ls_app_username, ls_app_password

In [ ]:
resp_app = ls_nb_utils.get_access_token(ls_app_username, ls_app_password, 'jwt_login', ls_app_url)
resp_app

In [ ]:
token=resp_app['token']

In [ ]:
%%time
params = {"searchScope": "FRIEND"}
params = {}
res = ls_nb_utils.test_ls_post(params, 'place/search', url = ls_app_url, token = token, target = 'packet')

In [ ]:
places = res['foundPlaces']
len(places)

In [ ]:
places[1]['place']

In [ ]:
places[1]['userStories'][0]

In [ ]:
def make_stories(pl):
    user_stories = pl['userStories']
    location =  pl['place']['locationString']
    user_st_text = f'A number of tourists visited {location} and left their stories.\n'
    for us in user_stories:
        user = us['user']['username']
        story = ''
        for st in us['stories']:
            t = st['timestamp']
            s = st['story']
            if t:
                story += f' on {t} '
            story += s                      
        user_st_text += f'Visitor {user} told the following stories: {story}' + '\n-------------------\n'
    return user_st_text
s = make_stories(places[0])
print(s)

In [ ]:
def select_places(places, value, by = 'user', logic = 'eq' ):
    ret = []
    match by:
        case 'user':
            for pl in places:
                user_stories = pl['userStories']
                for us in user_stories:
                    user = us['user']['username']
                    match logic:
                        case 'eq':
                            if user == value:
                                ret.append(pl)
                        case 'neq':
                            if user != value:
                                ret.append(pl)
                                
        case _:
            ret = []
    return ret

sel_places = select_places(places, 'inseo', by = 'user', logic = 'eq')    

In [ ]:
len(sel_places), len(places)

In [ ]:
def make_all_stories(places):
    all_stories = ''
    for pl in places:
        all_stories += make_stories(pl)
    return all_stories

mb_stories = make_all_stories(sel_places)
print(mb_stories)

In [ ]:
sel_places[0]